## IMPORT

In [775]:
import pandas as pd 
import numpy as np
from pytrends.request import TrendReq
from pytrends import dailydata
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [776]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


## 각 자산별 DATA

In [777]:
Ndata = pd.read_csv("nas.csv")
Gdata = pd.read_csv("gold.csv")
Odata = pd.read_csv("oildpb.csv")
Bdata = pd.read_csv("bitcoin.csv")
Bndata = pd.read_csv("bnb.csv")
Rdata = pd.read_csv("xrp.csv")
Tdata = pd.read_csv("tron.csv")
Sdata = pd.read_csv("sbi.csv")
Ddata = pd.read_csv("doge.csv")
Edata = pd.read_csv("eth.csv")
Adata = pd.read_csv("ada.csv")
Sdata

,Date,Price,Open,High,Low,Vol.,Change %
0,May 12 2021,0.000018,0.000030,0.000030,0.000018,86848.92B,-42.00%
1,May 13 2021,0.000021,0.000018,0.000021,0.000016,91382.21B,18.97%
2,May 14 2021,0.000018,0.000021,0.000021,0.000018,40929.23B,-12.14%
3,May 15 2021,0.000016,0.000018,0.000018,0.000015,43724.10B,-15.87%
4,May 16 2021,0.000018,0.000016,0.000018,0.000013,61614.36B,14.23%
...,...,...,...,...,...,...,...
384,May 31 2022,0.000012,0.000012,0.000012,0.000012,5805.49B,-3.62%
385,Jun 01 2022,0.000011,0.000012,0.000012,0.000011,5923.04B,-7.16%
386,Jun 02 2022,0.000011,0.000011,0.000011,0.000011,3451.79B,2.94%
387,Jun 03 2022,0.000011,0.000011,0.000011,0.000011,3610.97B,-3.66%


In [779]:
oil = Odata.set_index("DATE")
oil.columns = ['OIL']
oil.index = pd.to_datetime(oil.index)
oil = oil.sort_index()

In [780]:
bnb = pd.DataFrame(Bndata.iloc[:,[0,1]]).set_index("DATE")
bnb.columns = ['BNB']

bnb.index = pd.to_datetime(bnb.index)
bnb = bnb.sort_index()
pbnb = bnb.pct_change().dropna()

In [781]:
xrp = pd.DataFrame(Rdata.iloc[:,[0,1]]).set_index("DATE")
xrp.columns = ["XRP"]
xrp.index = pd.to_datetime(xrp.index)
xrp = xrp.sort_index()
pxrp = xrp.pct_change().dropna()

In [782]:
btc = pd.DataFrame(Bdata.iloc[:,[0,1]]).set_index("DATE")
btc.columns = ["BTC"]
btc.index = pd.to_datetime(btc.index)
btc = btc.sort_index()
pbtc = btc.pct_change().dropna()

In [783]:
tron = pd.DataFrame(Tdata.iloc[:,[0,1]]).set_index("DATE")
tron.columns = ["TRON"]
tron.index = pd.to_datetime(tron.index)
tron = tron.sort_index()
ptron = tron.pct_change().dropna()

In [784]:
sbi = pd.DataFrame(Sdata.iloc[:,[0,1]]).set_index("Date")
sbi.columns = ["SHIB"]
sbi.index = pd.to_datetime(sbi.index)
sbi = sbi.sort_index()
psbi = sbi.pct_change().dropna()

In [785]:
doge = pd.DataFrame(Ddata.iloc[:,[0,1]]).set_index("Date")
doge.columns = ["DOGE"]
doge.index = pd.to_datetime(doge.index)
doge = doge.sort_index()
pdoge = doge.pct_change().dropna()

In [786]:
eth = pd.DataFrame(Edata.iloc[:,[0,1]]).set_index("Date")
eth.columns = ["ETH"]
eth.index = pd.to_datetime(eth.index)
eth = eth.sort_index()
peth = eth.pct_change().dropna()

In [787]:
ada = pd.DataFrame(Adata.iloc[:,[0,1]]).set_index("Date")
ada.columns = ["ADA"]
ada.index = pd.to_datetime(ada.index)
ada = ada.sort_index()
pada = ada.pct_change().dropna()

In [788]:
gold = pd.DataFrame(Gdata.iloc[:,0:2]).set_index("DATE")
gold.columns = ["GOLD"]
gold.index = pd.to_datetime(gold.index)
gold = gold.sort_index()
pgold = gold.pct_change().dropna()

In [789]:
nas = pd.DataFrame(Ndata.iloc[:,0:2]).set_index("DATE")
nas.columns = ["NASDAQ"]
nas.index = pd.to_datetime(nas.index)
nas = nas.sort_index()
pnas = nas.pct_change().dropna()

In [790]:
idx = oil[oil["OIL"] == "."].index
oil.drop(idx, inplace = True)
oil.columns = ["OILPRICE"]
foil = oil.astype(float)
poil = foil.pct_change().dropna()

In [791]:
df = pd.concat([pnas, pgold, poil, pbtc, peth, pbnb, pxrp, pada, ptron, pdoge, psbi], join='outer', axis = 1)

## 

## 비트코인과 나스닥의 상관관계
 - t-value는 유의함을 보여주지만 상관관계는 0.41정도로 낮게 나타나기 때문에 공선성문제가 생기지 않을 것으로 판단하여 이후 OLS에서 둘 다 독립변수로 사용하였음

In [793]:
df_n = df.loc[:, ["BTC", "NASDAQ"]].dropna()
y_n = df_n.loc[:, ["BTC"]].dropna()
x_n = df_n.loc[:, ["NASDAQ"]].dropna()
x_n = sm.add_constant(x_n)
df.loc[:, ["BTC", "NASDAQ"]].corr()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,BTC,NASDAQ
BTC,1.000000,0.411842
NASDAQ,0.411842,1.000000


In [794]:
nas_model = sm.OLS(y_n, x_n).fit().summary()
nas_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BTC   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     123.4
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           3.28e-26
Time:                        23:06:40   Log-Likelihood:                 1085.1
No. Observations:                 606   AIC:                            -2166.
Df Residuals:                     604   BIC:                            -2157.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0027      0.002      1.633      0.103      -0.001       0.006
NASDAQ         0.9898      0.089     11.107      0.000       0.815       1.165
==============================================================================
Omnibus:                       97.473   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              906.935
Skew:                          -0.364   Prob(JB):                    1.15e-197
Kurtosis:                       8.949   Cond. No.                         54.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [795]:
df_v = df.loc[:, ['NASDAQ', 'GOLD', 'OILPRICE', 'BTC']]
df_v.corr()

,NASDAQ,GOLD,OILPRICE,BTC
NASDAQ,1.000000,0.113950,0.102502,0.411842
GOLD,0.113950,1.000000,0.030745,0.111988
OILPRICE,0.102502,0.030745,1.000000,0.078313
BTC,0.411842,0.111988,0.078313,1.000000


## 각 코인별 데일리 트렌드 지수

In [796]:
daily_btc = pd.read_csv("daily_btc.csv")
daily_eth = pd.read_csv("daily_eth.csv")
daily_bnb = pd.read_csv("daily_bnb.csv")
daily_xrp = pd.read_csv("daily_xrp.csv")
daily_ada = pd.read_csv("daily_ada.csv")
daily_trn = pd.read_csv("daily_trn.csv")
daily_dog = pd.read_csv("daily_dog.csv")
daily_shb = pd.read_csv("daily_shb.csv")

In [797]:
dbtc = pd.DataFrame(daily_btc.iloc[:,1:3]).set_index("date")
dbtc.columns = ["daily_BTC"]
dbtc.index = pd.to_datetime(dbtc.index)
dbtc = dbtc.sort_index()

deth = pd.DataFrame(daily_eth.iloc[:,1:3]).set_index("date")
deth.columns = ["daily_ETH"]
deth.index = pd.to_datetime(deth.index)
deth = deth.sort_index()

dbnb = pd.DataFrame(daily_bnb.iloc[:,1:3]).set_index("date")
dbnb.columns = ["daily_BNB"]
dbnb.index = pd.to_datetime(dbnb.index)
dbnb = dbnb.sort_index()

dxrp = pd.DataFrame(daily_xrp.iloc[:,1:3]).set_index("date")
dxrp.columns = ["daily_XRP"]
dxrp.index = pd.to_datetime(dxrp.index)
dxrp = dxrp.sort_index()

dada = pd.DataFrame(daily_ada.iloc[:,1:3]).set_index("date")
dada.columns = ["daily_ADA"]
dada.index = pd.to_datetime(dada.index)
dada = dada.sort_index()

dtrn = pd.DataFrame(daily_trn.iloc[:,1:3]).set_index("date")
dtrn.columns = ["daily_TRON"]
dtrn.index = pd.to_datetime(dtrn.index)
dtrn = dtrn.sort_index()

ddog = pd.DataFrame(daily_dog.iloc[:,1:3]).set_index("date")
ddog.columns = ["daily_DOGE"]
ddog.index = pd.to_datetime(ddog.index)
ddog = ddog.sort_index()

dsbi = pd.DataFrame(daily_shb.iloc[:,1:3]).set_index("date")
dsbi.columns = ["daily_SHIB"]
dsbi.index = pd.to_datetime(dsbi.index)
dsbi = dsbi.sort_index()

In [798]:
dbtc = dbtc.pct_change()
deth = deth.pct_change()
dbnb = dbnb.pct_change()
dxrp = dxrp.pct_change()
dada = dada.pct_change()
dtrn = dtrn.pct_change()
ddog = ddog.pct_change()
dsbi = dsbi.pct_change()

## 더미변수 만들기

In [800]:
df_eth = df.loc[:, ["ETH", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_eth = pd.concat([df_eth, deth], join = 'inner', axis = 1).dropna()
df_bnb = df.loc[:, ["BNB", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_bnb = pd.concat([df_bnb, dbnb], join = 'inner', axis = 1).dropna()
df_xrp = df.loc[:, ["XRP", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_xrp = pd.concat([df_xrp, dxrp], join = 'inner', axis = 1).dropna()
df_ada = df.loc[:, ["ADA", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_ada = pd.concat([df_ada, dada], join = 'inner', axis = 1).dropna()
df_trn = df.loc[:, ["TRON", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_trn = pd.concat([df_trn, dtrn], join = 'inner', axis = 1).dropna()
df_dog = df.loc[:, ["DOGE", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_dog = pd.concat([df_dog, ddog], join = 'inner', axis = 1).dropna()
df_sbi = df.loc[:, ["SHIB", "NASDAQ", "GOLD", "OILPRICE", "BTC"]].dropna()
df_sbi = pd.concat([df_sbi, dsbi], join = 'inner', axis = 1).dropna()

In [801]:
eth_d = df_eth["ETH"].apply(lambda x : True if x >= 0 else False)
eth_d = pd.get_dummies(pd.DataFrame(eth_d), columns = ["ETH"])
dd_eth = pd.concat([eth_d,deth], join='inner', axis=1).dropna()
dd_eth = dd_eth.loc[:,['ETH_False', 'ETH_True']].mul(dd_eth.loc[:,"daily_ETH"], axis = 0)

bnb_d = df_bnb["BNB"].apply(lambda x : True if x >= 0 else False)
bnb_d = pd.get_dummies(pd.DataFrame(bnb_d), columns = ["BNB"])
dd_bnb = pd.concat([bnb_d,dbnb], join='inner', axis=1).dropna()
dd_bnb = dd_bnb.loc[:,['BNB_False', 'BNB_True']].mul(dd_bnb.loc[:,"daily_BNB"], axis = 0)

xrp_d = df_xrp["XRP"].apply(lambda x : True if x >= 0 else False)
xrp_d = pd.get_dummies(pd.DataFrame(xrp_d), columns = ["XRP"])
dd_xrp = pd.concat([xrp_d,dxrp], join='inner', axis=1).dropna()
dd_xrp = dd_xrp.loc[:,['XRP_False', 'XRP_True']].mul(dd_xrp.loc[:,"daily_XRP"], axis = 0)

ada_d = df_ada["ADA"].apply(lambda x : True if x >= 0 else False)
ada_d = pd.get_dummies(pd.DataFrame(ada_d), columns = ["ADA"])
dd_ada = pd.concat([ada_d,dada], join='inner', axis=1).dropna()
dd_ada = dd_ada.loc[:,['ADA_False', 'ADA_True']].mul(dd_ada.loc[:,"daily_ADA"], axis = 0)

trn_d = df_trn["TRON"].apply(lambda x : True if x >= 0 else False)
trn_d = pd.get_dummies(pd.DataFrame(trn_d), columns = ["TRON"])
dd_trn = pd.concat([trn_d,dtrn], join='inner', axis=1).dropna()
dd_trn = dd_trn.loc[:,['TRON_False', 'TRON_True']].mul(dd_trn.loc[:,"daily_TRON"], axis = 0)

dog_d = df_dog["DOGE"].apply(lambda x : True if x >= 0 else False)
dog_d = pd.get_dummies(pd.DataFrame(dog_d), columns = ["DOGE"])
dd_dog = pd.concat([dog_d,ddog], join='inner', axis=1).dropna()
dd_dog = dd_dog.loc[:,['DOGE_False', 'DOGE_True']].mul(dd_dog.loc[:,"daily_DOGE"], axis = 0)

sbi_d = df_sbi["SHIB"].apply(lambda x : True if x >= 0 else False)
sbi_d = pd.get_dummies(pd.DataFrame(sbi_d), columns = ["SHIB"])
dd_sbi = pd.concat([sbi_d,dsbi], join='inner', axis=1).dropna()
dd_sbi = dd_sbi.loc[:,['SHIB_False', 'SHIB_True']].mul(dd_sbi.loc[:,"daily_SHIB"], axis = 0)

In [802]:
df_e = pd.concat([df_eth, dd_eth], join='inner', axis=1)
df_b = pd.concat([df_bnb, dd_bnb], join='inner', axis=1)
df_x = pd.concat([df_xrp, dd_xrp], join='inner', axis=1)
df_a = pd.concat([df_ada, dd_ada], join='inner', axis=1)
df_t = pd.concat([df_trn, dd_trn], join='inner', axis=1)
df_d = pd.concat([df_dog, dd_dog], join='inner', axis=1)
df_s = pd.concat([df_sbi, dd_sbi], join='inner', axis=1)

## 각각 OLS에 대하여 독립변수, 종속변수 설정

In [803]:
y_e = df_e.loc[:, "ETH"].dropna()
x_e = df_e.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ETH_False", "ETH_True"]].dropna()
#x_e = df_e.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_ETH"]].dropna()
x_e = sm.add_constant(x_e)

y_b = df_b.loc[:, "BNB"].dropna()
x_b = df_b.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "BNB_False", "BNB_True"]].dropna()
#x_b = df_b.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_BNB"]].dropna()
x_b = sm.add_constant(x_b)

y_x = df_x.loc[:, "XRP"].dropna()
x_x = df_x.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "XRP_False", "XRP_True"]].dropna()
#x_x = df_x.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_XRP"]].dropna()
x_x = sm.add_constant(x_x)

y_a = df_a.loc[:, "ADA"].dropna()
x_a = df_a.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ADA_False", "ADA_True"]].dropna()
#x_a = df_a.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_ADA"]].dropna()
x_a = sm.add_constant(x_a)

y_t = df_t.loc[:, "TRON"].dropna()
x_t = df_t.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "TRON_False", "TRON_True"]].dropna()
#x_t = df_t.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_TRON"]].dropna()
x_t = sm.add_constant(x_t)

y_d = df_d.loc[:, "DOGE"].dropna()
x_d = df_d.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "DOGE_False", "DOGE_True"]].dropna()
#x_d = df_d.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_DOGE"]].dropna()
x_d = sm.add_constant(x_d)

y_s = df_s.loc[:, "SHIB"].dropna()
x_s = df_s.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "SHIB_False", "SHIB_True"]].dropna()
#x_s = df_s.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "daily_SHIB"]].dropna()
x_s = sm.add_constant(x_s)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## OLS 결과

In [804]:
eth_model = sm.OLS(y_e, x_e).fit().summary()
bnb_model = sm.OLS(y_b, x_b).fit().summary()
xrp_model = sm.OLS(y_x, x_x).fit().summary()
ada_model = sm.OLS(y_a, x_a).fit().summary()
trn_model = sm.OLS(y_t, x_t).fit().summary()
dog_model = sm.OLS(y_d, x_d).fit().summary()

In [805]:
eth_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     258.2
Date:                Tue, 07 Jun 2022   Prob (F-statistic):          9.44e-162
Time:                        23:06:40   Log-Likelihood:                 1265.7
No. Observations:                 600   AIC:                            -2517.
Df Residuals:                     593   BIC:                            -2487.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.001      0.407      0.684      -0.002       0.003
NASDAQ         0.2646      0.073      3.649      0.000       0.122       0.407
GOLD           0.1015      0.106      0.959      0.338      -0.106       0.309
OILPRICE       0.0015      0.008      0.179      0.858      -0.015       0.018
BTC            0.9830      0.031     32.145      0.000       0.923       1.043
ETH_False     -0.0211      0.008     -2.770      0.006      -0.036      -0.006
ETH_True       0.0237      0.007      3.510      0.000       0.010       0.037
==============================================================================
Omnibus:                       57.281   Durbin-Watson:                   2.096
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              178.849
Skew:                           0.425   Prob(JB):                     1.46e-39
Kurtosis:                       5.536   Cond. No.                         88.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [825]:
x_e_u = x_e.drop(['GOLD', 'OILPRICE'], axis = 'columns')
eth_model_update = sm.OLS(y_e, x_e_u).fit().summary()
eth_model_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     387.8
Date:                Tue, 07 Jun 2022   Prob (F-statistic):          3.92e-164
Time:                        23:15:05   Log-Likelihood:                 1265.2
No. Observations:                 600   AIC:                            -2520.
Df Residuals:                     595   BIC:                            -2498.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.001      0.433      0.665      -0.002       0.003
NASDAQ         0.2707      0.072      3.758      0.000       0.129       0.412
BTC            0.9851      0.030     32.341      0.000       0.925       1.045
ETH_False     -0.0212      0.008     -2.791      0.005      -0.036      -0.006
ETH_True       0.0238      0.007      3.526      0.000       0.011       0.037
==============================================================================
Omnibus:                       59.020   Durbin-Watson:                   2.096
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              182.367
Skew:                           0.445   Prob(JB):                     2.51e-40
Kurtosis:                       5.550   Cond. No.                         60.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [806]:
bnb_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     85.33
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           6.92e-77
Time:                        23:06:40   Log-Likelihood:                 942.51
No. Observations:                 600   AIC:                            -1871.
Df Residuals:                     593   BIC:                            -1840.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0043      0.002      2.029      0.043       0.000       0.008
NASDAQ         0.2133      0.124      1.714      0.087      -0.031       0.458
GOLD           0.0915      0.181      0.504      0.614      -0.265       0.448
OILPRICE       0.0026      0.015      0.180      0.857      -0.026       0.031
BTC            0.9926      0.051     19.275      0.000       0.891       1.094
BNB_False      0.0813      0.033      2.448      0.015       0.016       0.146
BNB_True       0.0464      0.027      1.736      0.083      -0.006       0.099
==============================================================================
Omnibus:                      556.841   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36538.542
Skew:                           3.811   Prob(JB):                         0.00
Kurtosis:                      40.463   Cond. No.                         88.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [827]:
x_b_u = x_b.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'BNB_True'], axis = 'columns')
bnb_model_update = sm.OLS(y_b, x_b_u).fit().summary()
bnb_model_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.458
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     252.0
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           4.44e-80
Time:                        23:15:50   Log-Likelihood:                 939.43
No. Observations:                 600   AIC:                            -1873.
Df Residuals:                     597   BIC:                            -1860.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0041      0.002      1.929      0.054   -7.39e-05       0.008
BTC            1.0319      0.047     22.045      0.000       0.940       1.124
BNB_False      0.0816      0.033      2.471      0.014       0.017       0.146
==============================================================================
Omnibus:                      574.122   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40910.103
Skew:                           3.981   Prob(JB):                         0.00
Kurtosis:                      42.661   Cond. No.                         22.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [807]:
xrp_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.472
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     88.45
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           4.89e-79
Time:                        23:06:40   Log-Likelihood:                 934.32
No. Observations:                 600   AIC:                            -1855.
Df Residuals:                     593   BIC:                            -1824.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0036      0.002     -1.670      0.095      -0.008       0.001
NASDAQ         0.0757      0.126      0.600      0.549      -0.172       0.324
GOLD          -0.1505      0.184     -0.819      0.413      -0.511       0.210
OILPRICE      -0.0046      0.015     -0.312      0.755      -0.034       0.024
BTC            0.8994      0.053     17.057      0.000       0.796       1.003
XRP_False     -0.0410      0.013     -3.095      0.002      -0.067      -0.015
XRP_True       0.0548      0.006      8.831      0.000       0.043       0.067
==============================================================================
Omnibus:                      145.578   Durbin-Watson:                   2.162
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3725.512
Skew:                           0.403   Prob(JB):                         0.00
Kurtosis:                      15.181   Cond. No.                         88.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [828]:
x_x_u = x_x.drop(['NASDAQ', 'GOLD', 'OILPRICE'], axis = 'columns')
xrp_model_update = sm.OLS(y_x, x_x_u).fit().summary()
xrp_model_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.471
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     177.1
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           4.26e-82
Time:                        23:18:02   Log-Likelihood:                 933.79
No. Observations:                 600   AIC:                            -1860.
Df Residuals:                     596   BIC:                            -1842.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0036      0.002     -1.695      0.091      -0.008       0.001
BTC            0.9067      0.048     18.877      0.000       0.812       1.001
XRP_False     -0.0416      0.013     -3.153      0.002      -0.067      -0.016
XRP_True       0.0549      0.006      8.868      0.000       0.043       0.067
==============================================================================
Omnibus:                      146.748   Durbin-Watson:                   2.158
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3742.395
Skew:                           0.417   Prob(JB):                         0.00
Kurtosis:                      15.207   Cond. No.                         23.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [808]:
ada_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     94.26
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           6.14e-83
Time:                        23:06:40   Log-Likelihood:                 965.88
No. Observations:                 600   AIC:                            -1918.
Df Residuals:                     593   BIC:                            -1887.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007      0.002      0.304      0.762      -0.004       0.005
NASDAQ         0.3774      0.120      3.152      0.002       0.142       0.613
GOLD          -0.1002      0.175     -0.573      0.567      -0.443       0.243
OILPRICE      -0.0010      0.014     -0.071      0.943      -0.029       0.027
BTC            0.9407      0.050     18.697      0.000       0.842       1.039
ADA_False     -0.0327      0.015     -2.195      0.029      -0.062      -0.003
ADA_True       0.0413      0.015      2.716      0.007       0.011       0.071
==============================================================================
Omnibus:                      216.641   Durbin-Watson:                   2.115
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1614.229
Skew:                           1.399   Prob(JB):                         0.00
Kurtosis:                      10.533   Cond. No.                         88.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [829]:
x_a_u = x_a.drop(['GOLD', 'OILPRICE'], axis = 'columns')
ada_model_update = sm.OLS(y_a, x_a_u).fit().summary()
ada_model_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.484
Method:                 Least Squares   F-statistic:                     141.7
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           5.10e-85
Time:                        23:19:30   Log-Likelihood:                 965.71
No. Observations:                 600   AIC:                            -1921.
Df Residuals:                     595   BIC:                            -1899.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.002      0.296      0.767      -0.004       0.005
NASDAQ         0.3720      0.119      3.127      0.002       0.138       0.606
BTC            0.9389      0.050     18.727      0.000       0.840       1.037
ADA_False     -0.0326      0.015     -2.195      0.029      -0.062      -0.003
ADA_True       0.0407      0.015      2.688      0.007       0.011       0.071
==============================================================================
Omnibus:                      216.399   Durbin-Watson:                   2.116
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1618.595
Skew:                           1.395   Prob(JB):                         0.00
Kurtosis:                      10.547   Cond. No.                         61.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [809]:
trn_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.478
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                     90.55
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.84e-80
Time:                        23:06:40   Log-Likelihood:                 1004.0
No. Observations:                 600   AIC:                            -1994.
Df Residuals:                     593   BIC:                            -1963.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0014      0.002      0.733      0.464      -0.002       0.005
NASDAQ         0.0776      0.112      0.690      0.490      -0.143       0.298
GOLD           0.1105      0.164      0.673      0.501      -0.212       0.433
OILPRICE      -0.0006      0.013     -0.049      0.961      -0.026       0.025
BTC            0.9490      0.047     20.368      0.000       0.857       1.041
TRON_False     0.0203      0.016      1.299      0.194      -0.010       0.051
TRON_True      0.0642      0.014      4.585      0.000       0.037       0.092
==============================================================================
Omnibus:                      225.373   Durbin-Watson:                   2.221
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2268.552
Skew:                           1.366   Prob(JB):                         0.00
Kurtosis:                      12.126   Cond. No.                         88.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [830]:
x_t_u = x_t.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'TRON_False'], axis = 'columns')
tron_model_update = sm.OLS(y_t, x_t_u).fit().summary()
tron_model_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     270.9
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.89e-84
Time:                        23:20:51   Log-Likelihood:                 1002.6
No. Observations:                 600   AIC:                            -1999.
Df Residuals:                     597   BIC:                            -1986.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.002      0.580      0.562      -0.003       0.005
BTC            0.9689      0.042     23.037      0.000       0.886       1.052
TRON_True      0.0646      0.014      4.623      0.000       0.037       0.092
==============================================================================
Omnibus:                      228.070   Durbin-Watson:                   2.214
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2288.107
Skew:                           1.388   Prob(JB):                         0.00
Kurtosis:                      12.155   Cond. No.                         22.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [810]:
dog_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DOGE   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.2025
Date:                Tue, 07 Jun 2022   Prob (F-statistic):              0.976
Time:                        23:06:40   Log-Likelihood:                -1408.1
No. Observations:                 600   AIC:                             2830.
Df Residuals:                     593   BIC:                             2861.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0970      0.105      0.922      0.357      -0.110       0.304
NASDAQ         0.0501      6.263      0.008      0.994     -12.249      12.350
GOLD           4.4398      9.120      0.487      0.627     -13.471      22.351
OILPRICE      -0.0327      0.732     -0.045      0.964      -1.471       1.406
BTC            0.5954      2.604      0.229      0.819      -4.520       5.711
DOGE_False    -0.0370      0.533     -0.069      0.945      -1.083       1.009
DOGE_True      0.1325      0.144      0.921      0.357      -0.150       0.415
==============================================================================
Omnibus:                     1514.850   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8787771.470
Skew:                          24.308   Prob(JB):                         0.00
Kurtosis:                     593.887   Cond. No.                         88.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2022~

#### NASDAQ vs BTC

In [811]:
df_n2 = df.loc['2022-01-01':, ["BTC", "NASDAQ"]].dropna()
y_n2 = df_n2.loc[:, ["BTC"]].dropna()
x_n2 = df_n2.loc[:, ["NASDAQ"]].dropna()
x_n2 = sm.add_constant(x_n2)
df_n2.loc[:, ["BTC", "NASDAQ"]].corr()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,BTC,NASDAQ
BTC,1.000000,0.563674
NASDAQ,0.563674,1.000000


In [812]:
df_v2 = df.loc['2022-01-01':, ['NASDAQ', 'GOLD', 'OILPRICE', 'BTC']]
df_v2.corr()

,NASDAQ,GOLD,OILPRICE,BTC
NASDAQ,1.000000,-0.217113,-0.061262,0.563674
GOLD,-0.217113,1.000000,0.463942,-0.098298
OILPRICE,-0.061262,0.463942,1.000000,0.071313
BTC,0.563674,-0.098298,0.071313,1.000000


In [813]:
nas_model2 = sm.OLS(y_n2, x_n2).fit().summary()
nas_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BTC   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     46.57
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           6.90e-10
Time:                        23:06:41   Log-Likelihood:                 205.55
No. Observations:                 102   AIC:                            -407.1
Df Residuals:                     100   BIC:                            -401.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.003     -0.304      0.762      -0.007       0.005
NASDAQ         1.0291      0.151      6.824      0.000       0.730       1.328
==============================================================================
Omnibus:                       26.461   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               63.422
Skew:                           0.930   Prob(JB):                     1.69e-14
Kurtosis:                       6.386   Cond. No.                         46.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### ALT vs BTC

In [814]:
df_e2 = df_e.loc['2022-01-01':]
df_b2 = df_b.loc['2022-01-01':]
df_x2 = df_x.loc['2022-01-01':]
df_a2 = df_a.loc['2022-01-01':]
df_t2 = df_t.loc['2022-01-01':]
df_d2 = df_d.loc['2022-01-01':]
df_s2 = df_s.loc['2022-01-01':]

In [815]:
y_e2 = df_e2.loc[:, "ETH"].dropna()
x_e2 = df_e2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ETH_False", "ETH_True"]].dropna()
x_e2 = sm.add_constant(x_e2)

y_b2 = df_b2.loc[:, "BNB"].dropna()
x_b2 = df_b2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "BNB_False", "BNB_True"]].dropna()
x_b2 = sm.add_constant(x_b2)

y_x2 = df_x2.loc[:, "XRP"].dropna()
x_x2 = df_x2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "XRP_False", "XRP_True"]].dropna()
x_x2 = sm.add_constant(x_x2)

y_a2 = df_a2.loc[:, "ADA"].dropna()
x_a2 = df_a2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "ADA_False", "ADA_True"]].dropna()
x_a2 = sm.add_constant(x_a2)

y_t2 = df_t2.loc[:, "TRON"].dropna()
x_t2 = df_t2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "TRON_False", "TRON_True"]].dropna()
x_t2 = sm.add_constant(x_t2)

y_d2 = df_d2.loc[:, "DOGE"].dropna()
x_d2 = df_d2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "DOGE_False", "DOGE_True"]].dropna()
x_d2 = sm.add_constant(x_d2)

y_s2 = df_s2.loc[:, "SHIB"].dropna()
x_s2 = df_s2.loc[:, ["NASDAQ", "GOLD", "OILPRICE", "BTC", "SHIB_False", "SHIB_True"]].dropna()
x_s2 = sm.add_constant(x_s2)

eth_model2 = sm.OLS(y_e2, x_e2).fit().summary()
bnb_model2 = sm.OLS(y_b2, x_b2).fit().summary()
xrp_model2 = sm.OLS(y_x2, x_x2).fit().summary()
ada_model2 = sm.OLS(y_a2, x_a2).fit().summary()
trn_model2 = sm.OLS(y_t2, x_t2).fit().summary()
dog_model2 = sm.OLS(y_d2, x_d2).fit().summary()
sbi_model2 = sm.OLS(y_s2, x_s2).fit().summary()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [816]:
eth_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     102.3
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           2.96e-38
Time:                        23:06:41   Log-Likelihood:                 262.65
No. Observations:                  98   AIC:                            -511.3
Df Residuals:                      91   BIC:                            -493.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.002      0.318      0.751      -0.003       0.004
NASDAQ         0.4492      0.106      4.222      0.000       0.238       0.660
GOLD           0.1250      0.205      0.610      0.543      -0.282       0.532
OILPRICE       0.0060      0.057      0.105      0.917      -0.108       0.120
BTC            0.9005      0.058     15.465      0.000       0.785       1.016
ETH_False     -0.0325      0.014     -2.248      0.027      -0.061      -0.004
ETH_True      -0.0082      0.013     -0.642      0.523      -0.034       0.017
==============================================================================
Omnibus:                        3.135   Durbin-Watson:                   2.105
Prob(Omnibus):                  0.209   Jarque-Bera (JB):                2.818
Skew:                          -0.187   Prob(JB):                        0.244
Kurtosis:                       3.742   Cond. No.                         119.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [831]:
x_e2_u = x_e2.drop(['GOLD', 'OILPRICE', 'ETH_True'], axis = 'columns')
eth_model2_update = sm.OLS(y_e2, x_e2_u).fit().summary()
eth_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ETH   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                     209.1
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.84e-41
Time:                        23:22:28   Log-Likelihood:                 262.19
No. Observations:                  98   AIC:                            -516.4
Df Residuals:                      94   BIC:                            -506.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.002      0.294      0.770      -0.003       0.004
NASDAQ         0.4400      0.102      4.317      0.000       0.238       0.642
BTC            0.8942      0.056     16.013      0.000       0.783       1.005
ETH_False     -0.0349      0.014     -2.481      0.015      -0.063      -0.007
==============================================================================
Omnibus:                        3.004   Durbin-Watson:                   2.140
Prob(Omnibus):                  0.223   Jarque-Bera (JB):                2.366
Skew:                          -0.285   Prob(JB):                        0.306
Kurtosis:                       3.504   Cond. No.                         63.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [817]:
bnb_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     39.08
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           3.81e-23
Time:                        23:06:41   Log-Likelihood:                 226.30
No. Observations:                  98   AIC:                            -438.6
Df Residuals:                      91   BIC:                            -420.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001      0.003      0.054      0.957      -0.005       0.005
NASDAQ         0.1247      0.154      0.813      0.419      -0.180       0.430
GOLD           0.0043      0.287      0.015      0.988      -0.567       0.575
OILPRICE       0.1176      0.085      1.390      0.168      -0.050       0.286
BTC            0.9171      0.080     11.434      0.000       0.758       1.076
BNB_False      0.0300      0.048      0.628      0.532      -0.065       0.125
BNB_True      -0.0383      0.046     -0.829      0.410      -0.130       0.053
==============================================================================
Omnibus:                       14.732   Durbin-Watson:                   2.449
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               27.616
Skew:                          -0.569   Prob(JB):                     1.01e-06
Kurtosis:                       5.339   Cond. No.                         116.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [832]:
x_b2_u = x_b2.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'BNB_False', 'BNB_True'], axis = 'columns')
bnb_model2_update = sm.OLS(y_b2, x_b2_u).fit().summary()
bnb_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BNB   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     229.9
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           3.21e-27
Time:                        23:23:46   Log-Likelihood:                 223.74
No. Observations:                  98   AIC:                            -443.5
Df Residuals:                      96   BIC:                            -438.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0006      0.003      0.254      0.800      -0.004       0.006
BTC            0.9612      0.063     15.161      0.000       0.835       1.087
==============================================================================
Omnibus:                       10.992   Durbin-Watson:                   2.435
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               19.835
Skew:                          -0.400   Prob(JB):                     4.93e-05
Kurtosis:                       5.054   Cond. No.                         25.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [818]:
xrp_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.582
Method:                 Least Squares   F-statistic:                     23.55
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.27e-16
Time:                        23:06:41   Log-Likelihood:                 195.55
No. Observations:                  98   AIC:                            -377.1
Df Residuals:                      91   BIC:                            -359.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0037      0.004     -1.034      0.304      -0.011       0.003
NASDAQ         0.2949      0.211      1.400      0.165      -0.124       0.713
GOLD          -0.1474      0.401     -0.367      0.714      -0.944       0.650
OILPRICE       0.0747      0.117      0.640      0.524      -0.157       0.307
BTC            0.8204      0.112      7.319      0.000       0.598       1.043
XRP_False     -0.0200      0.029     -0.688      0.493      -0.078       0.038
XRP_True       0.0725      0.023      3.152      0.002       0.027       0.118
==============================================================================
Omnibus:                       23.842   Durbin-Watson:                   2.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               86.643
Skew:                           0.665   Prob(JB):                     1.53e-19
Kurtosis:                       7.410   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [833]:
x_x2_u = x_x2.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'XRP_False'], axis = 'columns')
xrp_model2_update = sm.OLS(y_x2, x_x2_u).fit().summary()
xrp_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    XRP   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     69.46
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           2.58e-19
Time:                        23:24:57   Log-Likelihood:                 193.78
No. Observations:                  98   AIC:                            -381.6
Df Residuals:                      95   BIC:                            -373.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0041      0.004     -1.166      0.247      -0.011       0.003
BTC            0.9371      0.088     10.644      0.000       0.762       1.112
XRP_True       0.0677      0.023      3.003      0.003       0.023       0.112
==============================================================================
Omnibus:                       24.901   Durbin-Watson:                   2.196
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.970
Skew:                           0.711   Prob(JB):                     2.91e-20
Kurtosis:                       7.473   Cond. No.                         25.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [819]:
ada_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     36.38
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           3.74e-22
Time:                        23:06:41   Log-Likelihood:                 195.91
No. Observations:                  98   AIC:                            -377.8
Df Residuals:                      91   BIC:                            -359.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0029      0.004     -0.741      0.461      -0.011       0.005
NASDAQ         0.6646      0.211      3.151      0.002       0.246       1.084
GOLD          -0.0960      0.395     -0.243      0.808      -0.880       0.688
OILPRICE       0.1893      0.114      1.661      0.100      -0.037       0.416
BTC            0.9701      0.117      8.300      0.000       0.738       1.202
ADA_False     -0.0626      0.027     -2.302      0.024      -0.117      -0.009
ADA_True       0.0015      0.028      0.054      0.957      -0.054       0.057
==============================================================================
Omnibus:                       13.950   Durbin-Watson:                   2.127
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               34.698
Skew:                           0.394   Prob(JB):                     2.92e-08
Kurtosis:                       5.807   Cond. No.                         117.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [834]:
x_a2_u = x_a2.drop(['GOLD', 'OILPRICE', 'ADA_True'], axis = 'columns')
ada_model2_update = sm.OLS(y_a2, x_a2_u).fit().summary()
ada_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ADA   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     71.68
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           3.33e-24
Time:                        23:25:57   Log-Likelihood:                 194.28
No. Observations:                  98   AIC:                            -380.6
Df Residuals:                      94   BIC:                            -370.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0017      0.004     -0.473      0.637      -0.009       0.005
NASDAQ         0.6325      0.204      3.096      0.003       0.227       1.038
BTC            0.9929      0.109      9.068      0.000       0.776       1.210
ADA_False     -0.0711      0.027     -2.666      0.009      -0.124      -0.018
==============================================================================
Omnibus:                       18.124   Durbin-Watson:                   2.148
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.826
Skew:                           0.649   Prob(JB):                     2.25e-09
Kurtosis:                       5.840   Cond. No.                         63.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [820]:
trn_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     11.62
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.26e-09
Time:                        23:06:41   Log-Likelihood:                 181.17
No. Observations:                  98   AIC:                            -348.3
Df Residuals:                      91   BIC:                            -330.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.004      0.509      0.612      -0.006       0.010
NASDAQ         0.3869      0.245      1.581      0.117      -0.099       0.873
GOLD           0.5809      0.455      1.276      0.205      -0.324       1.485
OILPRICE       0.1781      0.131      1.355      0.179      -0.083       0.439
BTC            0.6734      0.128      5.251      0.000       0.419       0.928
TRON_False    -0.0010      0.035     -0.029      0.977      -0.070       0.068
TRON_True      0.0012      0.041      0.029      0.977      -0.079       0.082
==============================================================================
Omnibus:                       51.420   Durbin-Watson:                   2.466
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              200.874
Skew:                           1.724   Prob(JB):                     2.40e-44
Kurtosis:                       9.108   Cond. No.                         116.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [835]:
x_t2_u = x_t2.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'TRON_False', 'TRON_True'], axis = 'columns')
tron_model2_update = sm.OLS(y_t2, x_t2_u).fit().summary()
tron_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRON   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.380
Method:                 Least Squares   F-statistic:                     60.46
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           8.45e-12
Time:                        23:27:20   Log-Likelihood:                 177.23
No. Observations:                  98   AIC:                            -350.5
Df Residuals:                      96   BIC:                            -345.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0023      0.004      0.561      0.576      -0.006       0.010
BTC            0.7924      0.102      7.776      0.000       0.590       0.995
==============================================================================
Omnibus:                       45.378   Durbin-Watson:                   2.636
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              167.673
Skew:                           1.504   Prob(JB):                     3.89e-37
Kurtosis:                       8.658   Cond. No.                         25.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [821]:
dog_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DOGE   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     20.42
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           5.25e-15
Time:                        23:06:41   Log-Likelihood:                 182.08
No. Observations:                  98   AIC:                            -350.2
Df Residuals:                      91   BIC:                            -332.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0055      0.004     -1.342      0.183      -0.014       0.003
NASDAQ         0.2540      0.241      1.054      0.295      -0.225       0.733
GOLD          -0.3723      0.451     -0.825      0.411      -1.268       0.524
OILPRICE       0.0615      0.130      0.472      0.638      -0.197       0.320
BTC            0.8677      0.126      6.881      0.000       0.617       1.118
DOGE_False    -0.0125      0.038     -0.327      0.745      -0.089       0.064
DOGE_True      0.0780      0.017      4.576      0.000       0.044       0.112
==============================================================================
Omnibus:                       14.054   Durbin-Watson:                   2.519
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               45.951
Skew:                          -0.221   Prob(JB):                     1.05e-10
Kurtosis:                       6.325   Cond. No.                         116.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [836]:
x_d2_u = x_d2.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'DOGE_False'], axis = 'columns')
dog_model2_update = sm.OLS(y_d2, x_d2_u).fit().summary()
dog_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DOGE   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     61.14
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           8.57e-18
Time:                        23:28:45   Log-Likelihood:                 180.83
No. Observations:                  98   AIC:                            -355.7
Df Residuals:                      95   BIC:                            -347.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0058      0.004     -1.439      0.153      -0.014       0.002
BTC            0.9643      0.099      9.748      0.000       0.768       1.161
DOGE_True      0.0779      0.017      4.624      0.000       0.044       0.111
==============================================================================
Omnibus:                       14.804   Durbin-Watson:                   2.501
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               46.886
Skew:                          -0.299   Prob(JB):                     6.59e-11
Kurtosis:                       6.335   Cond. No.                         25.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [822]:
sbi_model2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   SHIB   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                     25.59
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           1.31e-17
Time:                        23:06:41   Log-Likelihood:                 171.78
No. Observations:                  98   AIC:                            -329.6
Df Residuals:                      91   BIC:                            -311.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0083      0.005     -1.810      0.074      -0.017       0.001
NASDAQ         0.5048      0.269      1.878      0.064      -0.029       1.039
GOLD          -0.1260      0.513     -0.245      0.807      -1.146       0.894
OILPRICE       0.0187      0.145      0.129      0.898      -0.270       0.307
BTC            0.9577      0.145      6.619      0.000       0.670       1.245
SHIB_False    -0.0271      0.020     -1.343      0.183      -0.067       0.013
SHIB_True      0.0415      0.008      5.038      0.000       0.025       0.058
==============================================================================
Omnibus:                       16.041   Durbin-Watson:                   2.691
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.885
Skew:                           0.113   Prob(JB):                     1.10e-15
Kurtosis:                       7.101   Cond. No.                         119.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [837]:
x_s2_u = x_s2.drop(['NASDAQ', 'GOLD', 'OILPRICE', 'SHIB_False'], axis = 'columns')
sbi_model2_update = sm.OLS(y_s2, x_s2_u).fit().summary()
sbi_model2_update

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   SHIB   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     73.11
Date:                Tue, 07 Jun 2022   Prob (F-statistic):           5.99e-20
Time:                        23:29:56   Log-Likelihood:                 169.01
No. Observations:                  98   AIC:                            -332.0
Df Residuals:                      95   BIC:                            -324.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0084      0.005     -1.857      0.066      -0.017       0.001
BTC            1.1596      0.112     10.311      0.000       0.936       1.383
SHIB_True      0.0386      0.008      4.838      0.000       0.023       0.054
==============================================================================
Omnibus:                       18.331   Durbin-Watson:                   2.644
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               94.964
Skew:                           0.131   Prob(JB):                     2.39e-21
Kurtosis:                       7.815   Cond. No.                         25.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""